## Tutorial showing use of a `Workspace` object
### Part 2: Switchboards
This tutorial introduces the `Switchboard` workspace object and demonstrates its use.  You may have gotten the sense from the last tutorial that screen real estate can quickly be taken up by plots and tables.  Wouldn't it me nice if we could interactively switch between plots or figures using buttons or sliders instead of having to scroll through endless pages of plots?  `Switchboard` to the rescue!

First though, let's run GST on the standard 1Q gate set to get some results (the same ones as the first tutorial).

In [1]:
import numpy as np
import pygsti
from pygsti.construction import std1Q_XYI

#The usual GST setup: we're going to run GST on the standard XYI 1-qubit gateset
gs_target = std1Q_XYI.gs_target
fiducials = std1Q_XYI.fiducials
germs = std1Q_XYI.germs
maxLengths = [1,2,4,8]
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(
    gs_target.gates.keys(), fiducials, fiducials, germs, maxLengths)

#Create some datasets for analysis
gs_datagen1 = gs_target.depolarize(gate_noise=0.1, spam_noise=0.001)
gs_datagen2 = gs_target.depolarize(gate_noise=0.05, spam_noise=0.01).rotate(rotate=0.01)

ds1 = pygsti.construction.generate_fake_data(gs_datagen1, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=1234)
ds2 = pygsti.construction.generate_fake_data(gs_datagen2, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=1234)
ds3 = ds1.copy_nonstatic(); ds3.add_counts_from_dataset(ds2); ds3.done_adding_data()

#Run GST on all three datasets
gs_target.set_all_parameterizations("TP")
results1 = pygsti.do_long_sequence_gst(ds1, gs_target, fiducials, fiducials, germs, maxLengths, verbosity=0)
results2 = pygsti.do_long_sequence_gst(ds2, gs_target, fiducials, fiducials, germs, maxLengths, verbosity=0)
results3 = pygsti.do_long_sequence_gst(ds3, gs_target, fiducials, fiducials, germs, maxLengths, verbosity=0)

#make some shorthand variable names for later
tgt = results1.gatesets['target']

ds1 = results1.dataset
ds2 = results2.dataset
ds3 = results3.dataset

gs1 = results1.gatesets['final estimate']
gs2 = results2.gatesets['final estimate']
gs3 = results3.gatesets['final estimate']

gss = results1.gatestring_structs['final']

Next we create the workspace, as before.  This time, we'll leave `autodisplay=False` (the default), to demonstrate how this gives us more control over when workspace items are displayed.  In particular, we'll build up a several workspace objects and display them all at once.

In [2]:
from pygsti.report import workspace
w = workspace.Workspace()  #create a new workspace
w.init_notebook_mode(connected=True) # and initialize it so it works within a notebook

Note that if we create a table it doesn't get displayed automatically.

In [3]:
tbl1 = w.GatesVsTargetTable(gs1, tgt)

To see it, we need to call `display()`:

In [4]:
tbl1.display()

Gate,Process Infidelity,1/2 Trace Distance,1/2 Diamond-Norm
Gi,0.074758,0.074836,0.074856
Gx,0.07094,0.070941,0.070941
Gy,0.076925,0.076929,0.076933


### Switchboards
A `Switchboard` is essentially a collection of one or more switches along with a dictionary of "values" which depend on some or all of the switch positions.  Each value looks like a NumPy `ndarray` whose axes correspond to the switches that value depends upon.  The array can hold whatever you want: `GateSet`s, `DataSet`s, `float`s, etc., and from the perspective of the plot and table workspace objects the value looks like the thing contained in its array (e.g. a *single* `GateSet`, `DataSet`, or `float`, etc.).  

Let's start off simple and create a switchboard with a single switch named "My Switch" that has two positions "On" and "Off":

In [5]:
switchbd = w.Switchboard(["My Switch"],[["On","Off"]],["buttons"])

Next, add a "value" to the switchboard called "gs" (for "gate set"), with is dependent on the 0-th (and only) switch of the switchboard:

In [6]:
switchbd.add("gs", [0])

Now `switchbd` has a member, `gs`, which looks like a 1-dimensional Numpy array (since `gs` only depends on a single switch) of length 2 (because that single switch has 2 positions). 

In [7]:
switchbd.gs.shape

(2,)

We'll use `switchbd.gs` to switch between the gate sets `gs1` and `gs2`.  We associate the "On" position with `gs1` and the "Off" position with `gs2` by simply assigning them to the corresponding locations of the array.  Note that we can use NumPy's fancy indexing to make this a breeze.

In [8]:
switchbd.gs[:] = [gs1,gs2]

Ok, now here's the magical part: even though `switchbd.gs` is really an array holding `GateSet` objects, when you provide it as an input to create a workspace item such as a plot or a table, it *behaves* like a single `GateSet` and can thus be used for any `GateSet`-type argument.  We'll use it as the first argument to `GatesVsTargetTable`. 

In [9]:
tbl2 = w.GatesVsTargetTable(switchbd.gs, tgt)

Note the the second argument (`tgt`, the target gate set) in the above call is just a plain old `GateSet`, just like it's always been up to this point.  The above line creates a table, `tbl2`, that is *connected* to the switchboard `switchbd`.  Let's display both the switchboard and the table together.  Note that currently, the **switchboard must be displayed *before* any dependent plots or tables**.

In [10]:
switchbd.display()
tbl2.display()

Gate,Process Infidelity,1/2 Trace Distance,1/2 Diamond-Norm
Gi,0.074758,0.074836,0.074856
Gx,0.07094,0.070941,0.070941
Gy,0.076925,0.076929,0.076933
Gate,Process Infidelity,1/2 Trace Distance,1/2 Diamond-Norm
Gi,0.038779,0.040832,0.040834
Gx,0.036812,0.037551,0.037555
Gy,0.037378,0.038203,0.038205


My pressing the "On" or "Off" button the table changes between displaying metrics for `gs1` vs. `tgt` and `gs2` vs. `tgt`, as expected.  In this simple example there was one switch controlling on table.  It is possible to have any number of switches controlling any number of tables and/or plots, and also to have multiple switchboards controlling a single plot or table.  In the following cells, more sophisticated uses of switchboards are demonstrated.

In [11]:
# Create a switchboard with straighforward dataset and gateset dropdown switches
switchbd2 = w.Switchboard(["dataset","gateset"], [["DS1","DS2","DS3"],["GS1","GS2","GS3"]], ["dropdown","dropdown"])
switchbd2.add("ds",(0,))
switchbd2.add("gs",(1,))
switchbd2.ds[:] = [ds1, ds2, ds3]
switchbd2.gs[:] = [gs1, gs2, gs3]

#Then create a chi2 plot that can show the goodness-of-fit between any gateset-dataset pair
gss2 = gss.copy(); gss2.germs = gss.germs[0:5] #truncate gss to only a subset of the germs
chi2plot = w.ColorBoxPlot(("chi2",), gss2, switchbd2.ds, switchbd2.gs, scale=0.75)

switchbd2.display()
chi2plot.display()

In [12]:
#Perform gauge optimizations of gs1 using different spam weights
spamWts = np.linspace(0.0,1.0,20)
gs_gaugeopts = [ pygsti.gaugeopt_to_target(gs1, tgt,{'gates': 1, 'spam': x}) for x in spamWts]

In [13]:
# Create a switchboard with a slider that controls the spam-weight used in gauge optimization
switchbd3 = w.Switchboard(["spam-weight"], [["%.2f" % x for x in spamWts]], ["slider"])
switchbd3.add("gsGO",(0,))
switchbd3.gsGO[:] = gs_gaugeopts

#Then create a comparison vs. target tables
tbl3 = w.GatesVsTargetTable(switchbd3.gsGO, tgt)
tbl4 = w.SpamVsTargetTable(switchbd3.gsGO, tgt)

switchbd3.display()
tbl3.display()
tbl4.display()

Gate,Process Infidelity,1/2 Trace Distance,1/2 Diamond-Norm
Gi,0.074758,0.074836,0.074856
Gx,0.07094,0.070941,0.070941
Gy,0.076925,0.076929,0.076933
Gate,Process Infidelity,1/2 Trace Distance,1/2 Diamond-Norm
Gi,0.074758,0.074836,0.074856
Gx,0.070939,0.070941,0.070941
Gy,0.076925,0.076929,0.076933
Gate,Process Infidelity,1/2 Trace Distance,1/2 Diamond-Norm
Gi,0.074758,0.074836,0.074856
Gx,0.070939,0.070941,0.070941


Prep/POVM,State Infidelity,1/2 Trace Distance
ρ0,0.000329,0.001948
E0,0.000863,0.004191
EC,-0.000335,0.004191
Prep/POVM,State Infidelity,1/2 Trace Distance
ρ0,0.000306,0.001873
E0,0.000845,0.004114
EC,-0.000312,0.004114
Prep/POVM,State Infidelity,1/2 Trace Distance
ρ0,0.000289,0.001805
E0,0.00083,0.004043


In [14]:
# Create a slider showing the color box plot at different GST iterations
switchbd4 = w.Switchboard(["max(L)"], [list(map(str,gss.Ls))], ["slider"])
switchbd4.add("gs",(0,))
switchbd4.add("gss",(0,))
switchbd4.gs[:] = results1.gatesets['iteration estimates']
switchbd4.gss[:] = results1.gatestring_structs['iteration']
            
#Then create a logl plot that can show the goodness-of-fit at different iterations
logLProgress = w.ColorBoxPlot(("logl",), switchbd4.gss, ds1, switchbd4.gs, scale=0.75)

switchbd4.display()
logLProgress.display()